# This notebook shows how to use Orchestrator APIs for user experiments

## Configure the Environment

In [ ]:
import os

# If you are using the FABRIC JupyterHub, the following three evnrionment vars
# were automatically provided when you logged in.
#os.environ['FABRIC_CREDMGR_HOST']='cm.fabric-testbed.net'
#os.environ['FABRIC_ORCHESTRATOR_HOST']='orchestrator.fabric-testbed.net'
#os.environ['FABRIC_TOKEN_LOCATION']=os.environ['HOME']+'/work/fabric_token.json'

# Bastion IPs
os.environ['FABRIC_BASTION_HOST'] = 'bastion-1.fabric-testbed.net'

# Set your Bastion username and private key
os.environ['FABRIC_BASTION_USERNAME']=<INSERT_YOUR_FABRIC_USERNAME>
os.environ['FABRIC_BASTION_KEY_LOCATION']=os.environ['HOME']+'/work/.ssh/id_rsa_fabric'

# Set the keypair FABRIC will install in your slice. 
os.environ['FABRIC_SLICE_PRIVATE_KEY_FILE']=os.environ['HOME']+'/.ssh/id_rsa'
os.environ['FABRIC_SLICE_PUBLIC_KEY_FILE']=os.environ['HOME']+'/.ssh/id_rsa.pub'

# If your slice private key uses a passphrase, set the passphrase
#from getpass import getpass
#print('Please input private key passphrase. Press enter for no passphrase.')
#os.environ['FABRIC_SLICE_PRIVATE_KEY_PASSPHRASE']=getpass()

## Setup the Experiment

#### Import FABRIC API

In [ ]:
import json
import traceback

from fabrictestbed_extensions.fablib.fablib import fablib

In [ ]:
try:
    available_resources = fablib.get_available_resources()
    print(f"Available Resources: {available_resources}")
    available_resources.draw()
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()

## Create Slice
In Release 1.0, user is expected to assign the IP addresses manually. Please use the example comands indicated below:

## Configure Slice Parameters



In [ ]:
slice_name_base = 'MySliceL2Net_TACC2'

#test_sites = [ 'TACC', 'MAX', 'UTAH', 'NCSA', 'MICH', 'WASH', 'DALL', 'SALT']
test_sites = [ 'TACC' ]

node_name_base = 'node'
network_name_base='net'

image = 'default_ubuntu_20'
cores = 2
ram = 8
disk = 10

node_count = 5

In [ ]:
import glob
import json

#aggregate_json_dir=os.environ['HOME']+"/work/git/jupyter-examples/fabric_examples/testing_and_debugging/aggregates/"
aggregate_json_dir="./aggregates/"

aggregates = {}
aggregate_files = glob.glob(aggregate_json_dir+"*.json")
for file_name in aggregate_files:
    print(f"Reading {file_name}")
    site_name = file_name.replace(aggregate_json_dir,"",1).split(".")[0]
    #print(f"{site_name}")
    with open(file_name, 'r') as infile:
        site_info = json.load(infile)
    #print(f"{site_json}")
    aggregates[site_name] = site_info

### Create the Slice

NIC component models options:
- NIC_Basic 
- NIC_ConnectX_5 
- NIC_ConnectX_6

In [ ]:
slice_name=f"{slice_name_base}"
print(f"Creating slice {slice_name}")

networks = []

try:
    slice = fablib.new_slice(slice_name)
    for site in test_sites:
        print(f"{site}")
        site_info = aggregates[site]
             
        for node_info in site_info['Nodes']:
            print(f"{node_info['Name']}")
            worker = node_info['Name']
            worker_short_name = worker.split(".")[0]
            print(f"worker: {worker}")
            
            net_name = f"{network_name_base}_{site}_{worker_short_name}"
            ifaces = []
            for node_num in range(node_count):
                node_name = f"{node_name_base}_{site}_{node_num}_{worker_short_name}"
                

                # Node1
                node = slice.add_node(name=node_name, site=site)
                node.set_capacities(cores=cores, ram=ram, disk=disk)
                node.set_image(image)
                iface = node.add_component(model='NIC_Basic', name=f"{node_name}_nic").get_interfaces()[0]
                #iface1.set_vlan(vlan=100)

                ifaces.append(iface)

            # Network
            net1 = slice.add_l2network(name=net_name, interfaces=ifaces)

    #Submit Slice Request
    slice.submit(wait_progress=True)
except Exception as e:
    print(f"Slice Fail: {e}")
    traceback.print_exc()


In [ ]:
import time

slice_name=f"{slice_name_base}"
            
try:
    print(f"Waiting for slice {slice_name} ")

    #Get Slice
    slice = fablib.get_slice(slice_name)

    #Wait for ssh to be active
    slice.wait_ssh(progress=True, timeout=600,interval=60)
    #time.sleep(30)
    
    #Run post boo config
    slice.post_boot_config(verbose=True)
except Exception as e:
    print(f"Slice Failed: {e}")
    traceback.print_exc()

In [ ]:
try:
    print(f"Waiting for slice {slice_name} ")
    print(json.dumps(slice.get_interface_map(), indent=4, sort_keys=True))
except:
    print("Error")

In [ ]:
try:
    for reservtion_id,notice in slice.get_notices().items():
        print(f"reservtion_id: {reservtion_id}, notice: {notice}")
except Exception as e:
    print(f"Exception: \n{e}")
    

## Get the Slice

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    print(f"Slice: {slice.get_name()}")
except Exception as e:
    print(f"Get Slices Fail: {e}")

### Test SSH

In [ ]:
slice_name=f"{slice_name_base}"
print(f"Getting slice {slice_name} ")
try: 
    slice = fablib.get_slice(slice_name)
    for node in slice.get_nodes():
        print(f"Node: {node.get_name()}, Host: {node.get_host()}, SSH Test: {node.test_ssh()}")
        
except Exception as e:
    print(f"Fail: {e}")

### Get Attributes

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    for node in slice.get_nodes():
        print("Node:")
        print(f"   Name              : {node.get_name()}")
        print(f"   Cores             : {node.get_cores()}")
        print(f"   RAM               : {node.get_ram()}")
        print(f"   Disk              : {node.get_disk()}")
        print(f"   Image             : {node.get_image()}")
        print(f"   Image Type        : {node.get_image_type()}")
        print(f"   Host              : {node.get_host()}")
        print(f"   Site              : {node.get_site()}")
        print(f"   Management IP     : {node.get_management_ip()}")
        print(f"   Reservation ID    : {node.get_reservation_id()}")
        print(f"   Reservation State : {node.get_reservation_state()}")
        print(f"   SSH Command       : {node.get_ssh_command()}")
        print(f"   Components        :  ")
        for component in node.get_components():
            print(f"      Name             : {component.get_name()}")
            print(f"      Details          : {component.get_details()}")
            print(f"      Disk (G)         : {component.get_disk()}")
            print(f"      Units            : {component.get_unit()}")
            print(f"      PCI Address      : {component.get_pci_addr()}")
            print(f"      Model            : {component.get_model()}")
            print(f"      Type             : {component.get_type()}") 
        print(f"   Interfaces        :  ")
        for interface in node.get_interfaces():
            print(f"       Name                : {interface.get_name()}")
            print(f"           Bandwidth           : {interface.get_bandwidth()}")
            print(f"           VLAN                : {interface.get_vlan()}")          
            print(f"           OS Interface        : {interface.get_os_interface()}")       

except Exception as e:
    print(f"Fail: {e}")

## Get the Nodes

Retrieve the node information and save the management IP address.


### Configure Node1

Configure the interface on on  node1.  

```
ip addr add 192.168.1.10/24 dev eth1
```

In [ ]:
try:
    node1 = slice.get_node(name=node1_name)        
    node1_iface = node1.get_interface(network_name=network_name)  
    node1_iface.set_ip(ip="192.168.1.10", cidr="24")
    
    stdout, stderr = node1.execute(f'ip addr list {node1_iface.get_os_interface()}')
    print (stdout)
except Exception as e:
    print(f"Error: {e}")

### Configure Node2

Configure the interface on on  node2.  

```
ip addr add 192.168.1.11/24 dev eth1
```

In [ ]:
try:
    node2 = slice.get_node(name=node2_name)        
    node2_iface = node2.get_interface(network_name=network_name)  
    node2_iface.set_ip(ip="192.168.1.11", cidr="24")
    
    stdout, stderr = node2.execute(f'ip addr list {node2_iface.get_os_interface()}')
    print (stdout)
except Exception as e:
    print(f"Error: {e}")

## Delete Slice

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    slice.delete()
except Exception as e:
    print(f"Fail: {e}")